In [ ]:
!pip install -r https://raw.githubusercontent.com/nishinishic/sca-1-cp/main/requirements.txt


In [ ]:
import requests
import os
from PIL import Image
from io import BytesIO
from google.colab import userdata

# Directory to save images and metadata locally in the Colab session
output_dir = '/content/IPFS_images'
os.makedirs(output_dir, exist_ok=True)

# Fetch JWT token securely from Colab secrets
JWT = userdata.get('JWT_TOKEN')  # Ensure 'JWT_TOKEN' is set in Colab secrets

# Function to fetch pinned files from IPFS
def fetch_pinned_files():
    url = 'https://api.pinata.cloud/data/pinList'
    headers = {
        'Authorization': f'Bearer {JWT}',
    }

    # Fetch the data
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print('Error fetching pinned files:', response.text)
        return []

    # Process response
    data = response.json()
    pinned_files = data.get('rows', [])
    active_files = [file for file in pinned_files if file.get('date_unpinned') is None]

    return active_files

# Function to download images from IPFS and save with metadata
def download_ipfs_images():
    files = fetch_pinned_files()

    for file in files:
        # Extract metadata
        metadata = file.get('metadata', {}).get('keyvalues', {})
        lat = metadata.get('latitude')
        lng = metadata.get('longitude')
        ipfs_hash = file.get('ipfs_pin_hash')
        name = file.get('metadata', {}).get('name', 'Unknown')
        date_pinned = file.get('date_pinned')

        if not ipfs_hash or lat is None or lng is None:
            continue  # Skip if necessary data is missing

        # Download image
        image_url = f'https://ipfs.io/ipfs/{ipfs_hash}'
        try:
            img_response = requests.get(image_url)
            img_response.raise_for_status()  # Check if the request was successful
            img = Image.open(BytesIO(img_response.content))

            # Create filename based on the IPFS hash
            filename = f"{output_dir}/{name.replace(' ', '_')}_{ipfs_hash}.png"
            img.save(filename)
            print(f"Saved image: {filename}")

            # Save metadata
            metadata_filename = f"{output_dir}/{name.replace(' ', '_')}_{ipfs_hash}_metadata.txt"
            with open(metadata_filename, 'w') as meta_file:
                meta_file.write(f"Name: {name}\n")
                meta_file.write(f"Latitude: {lat}\n")
                meta_file.write(f"Longitude: {lng}\n")
                meta_file.write(f"Date Pinned: {date_pinned}\n")
                meta_file.write(f"IPFS Hash: {ipfs_hash}\n")

            print(f"Saved metadata: {metadata_filename}")

        except requests.RequestException as e:
            print(f"Error downloading image from IPFS: {e}")

# Run the function to download images and save metadata
download_ipfs_images()


Here we initialize Google Earth Engine. Might need to replace the project with your own, authenticating into GEE from your google account.

In [ ]:
# Install the Earth Engine API
!pip install earthengine-api
!pip install geopy

# Authenticate Earth Engine (one-time setup for each Colab session)
import ee

# Authenticate and initialize Earth Engine with the specified project
ee.Authenticate()
ee.Initialize(project='mapsim-418614')


Getting Satellite images based on lat, long, and time for each ipfs image.

In [ ]:
import ee
from geopy.distance import geodesic
import os
from IPython.display import Image, display
from datetime import datetime, timedelta
from PIL import Image as PILImage  # Avoids confusion with IPython's Image



# Directory where IPFS images and metadata are stored
metadata_dir = '/content/IPFS_images'

# Function to create a bounding box around a point
def get_bounding_box(lat, lng, distance_km=5):
    point = ee.Geometry.Point([lng, lat])
    bbox = point.buffer(distance_km * 500).bounds()  # Creates a bounding box around the point
    return bbox

# Function to fetch and visualize satellite imagery from Earth Engine
def fetch_earth_engine_imagery(lat, lng, date, distance_km=5, date_range_days=7):
    # Define the bounding box around the point
    bbox = get_bounding_box(lat, lng, distance_km)  # Adjustable bounding box

    # Define start and end dates for a flexible search window
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    start_date = (date_obj - timedelta(days=date_range_days)).strftime("%Y-%m-%d")
    end_date = (date_obj + timedelta(days=date_range_days)).strftime("%Y-%m-%d")

    # Select Sentinel-2 imagery (or Landsat if preferred)
    collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(bbox) \
        .filterDate(start_date, end_date) \
        .sort('CLOUDY_PIXEL_PERCENTAGE')  # Sort by cloud cover

    # Get the closest image with the least cloud cover
    image = collection.first()
    if image is None:
        print(f"No available imagery for the date range {start_date} to {end_date} at lat: {lat}, lng: {lng}")
        return None

    # Clip the image to the bounding box
    image = image.clip(bbox)

    # Generate the URL for the RGB bands
    url = image.getThumbURL({
        'region': bbox,
        'dimensions': 512,
        'format': 'png',
        'bands': ['B4', 'B3', 'B2'],  # RGB bands
        'min': 0,
        'max': 3000
    })

    # Display the image in Colab
    display(Image(url=url))
    print(f"Satellite image URL for lat: {lat}, lng: {lng} within date range: {start_date} to {end_date}")

# Function to read latitude, longitude, and date from metadata files and fetch imagery
def fetch_imagery_for_all_ipfs_images(distance_km=5, date_range_days=7):
    # Loop through all metadata files in the directory
    for filename in os.listdir(metadata_dir):
        if filename.endswith('_metadata.txt'):
            metadata_path = os.path.join(metadata_dir, filename)

            # Read the metadata file
            with open(metadata_path, 'r') as meta_file:
                data = meta_file.readlines()

                # Parse latitude, longitude, and date from metadata
                lat = lng = date_pinned = None
                for line in data:
                    if 'Latitude' in line:
                        lat = float(line.split(': ')[1].strip())
                    elif 'Longitude' in line:
                        lng = float(line.split(': ')[1].strip())
                    elif 'Date Pinned' in line:
                        date_pinned = line.split(': ')[1].strip().split('T')[0]  # Use only date part

                if lat and lng and date_pinned:
                    # Fetch and display satellite imagery for each lat/lng/date
                    print(f"Fetching imagery for lat: {lat}, lng: {lng}, date: {date_pinned}")
                    fetch_earth_engine_imagery(lat, lng, date_pinned, distance_km, date_range_days)
                else:
                    print(f"Metadata incomplete for {filename}, skipping.")

# Run the function to fetch imagery for all IPFS images with adjustable bounding box and date range
fetch_imagery_for_all_ipfs_images(distance_km=5, date_range_days=7)

Getting satellite images, and saving them / showing them side by side. WIP needs better organizing.

In [ ]:
import ee
from geopy.distance import geodesic
import os
import requests
from IPython.display import Image, display, HTML
from datetime import datetime, timedelta
from PIL import Image as PILImage  # Avoids confusion with IPython's Image
from io import BytesIO


# Directory where IPFS images, metadata, and satellite images are stored
metadata_dir = '/content/IPFS_images'
satellite_output_dir = '/content/IPFS_satellite_images'
os.makedirs(satellite_output_dir, exist_ok=True)

# Function to create a bounding box around a point
def get_bounding_box(lat, lng, distance_km=5):
    point = ee.Geometry.Point([lng, lat])
    bbox = point.buffer(distance_km * 500).bounds()  # Creates a bounding box around the point
    return bbox

# Function to fetch and save satellite imagery
def fetch_and_save_satellite_imagery(lat, lng, date, ipfs_hash, distance_km=5, date_range_days=7):
    # Define the bounding box around the point
    bbox = get_bounding_box(lat, lng, distance_km)  # Adjustable bounding box

    # Define start and end dates for a flexible search window
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    start_date = (date_obj - timedelta(days=date_range_days)).strftime("%Y-%m-%d")
    end_date = (date_obj + timedelta(days=date_range_days)).strftime("%Y-%m-%d")

    # Select Sentinel-2 imagery (or Landsat if preferred)
    collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(bbox) \
        .filterDate(start_date, end_date) \
        .sort('CLOUDY_PIXEL_PERCENTAGE')  # Sort by cloud cover

    # Get the closest image with the least cloud cover
    image = collection.first()
    if image is None:
        print(f"No available imagery for the date range {start_date} to {end_date} at lat: {lat}, lng: {lng}")
        return None

    # Clip the image to the bounding box
    image = image.clip(bbox)

    # Generate the URL for the RGB bands
    url = image.getThumbURL({
        'region': bbox,
        'dimensions': 512,
        'format': 'png',
        'bands': ['B4', 'B3', 'B2'],  # RGB bands
        'min': 0,
        'max': 3000
    })

    # Download the satellite image to save it locally
    response = requests.get(url)
    satellite_image_path = os.path.join(satellite_output_dir, f"{ipfs_hash}_satellite.png")
    with open(satellite_image_path, 'wb') as f:
        f.write(response.content)

    print(f"Satellite image saved to: {satellite_image_path}")
    return satellite_image_path

# Function to fetch and display IPFS and satellite imagery side-by-side
def display_ipfs_and_satellite_images(metadata_path, ipfs_hash):
    # Display IPFS image
    ipfs_image_path = metadata_path.replace('_metadata.txt', '.png')
    ipfs_image = PILImage.open(ipfs_image_path)

    # Display Satellite image
    metadata = {}
    with open(metadata_path, 'r') as meta_file:
        for line in meta_file:
            key, value = line.strip().split(': ')
            metadata[key.lower()] = value.strip()
    lat, lng, date = float(metadata['latitude']), float(metadata['longitude']), metadata['date pinned'].split('T')[0]

    # Fetch and save satellite image
    satellite_image_path = fetch_and_save_satellite_imagery(lat, lng, date, ipfs_hash)
    satellite_image = PILImage.open(satellite_image_path)

    # Display both images side by side
    combined_image = PILImage.new('RGB', (ipfs_image.width + satellite_image.width, max(ipfs_image.height, satellite_image.height)))
    combined_image.paste(ipfs_image, (0, 0))
    combined_image.paste(satellite_image, (ipfs_image.width, 0))
    display(combined_image)

# Function to process all IPFS images and display both IPFS and satellite images side-by-side
def fetch_and_display_all_images(distance_km=5, date_range_days=7):
    for filename in os.listdir(metadata_dir):
        if filename.endswith('_metadata.txt'):
            metadata_path = os.path.join(metadata_dir, filename)
            ipfs_hash = filename.split('_')[1]  # Assuming hash is in filename

            print(f"\nProcessing IPFS image and metadata for IPFS hash: {ipfs_hash}")
            display_ipfs_and_satellite_images(metadata_path, ipfs_hash)

# Run the function to fetch and display both IPFS and satellite images side-by-side
fetch_and_display_all_images(distance_km=5, date_range_days=7)